<a href="https://colab.research.google.com/github/chiranjeet14/ML_Journey/blob/master/Hackerearth-Predict_condition_and_insurance_amount/vehicle_insurance_image_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import gc
import time
import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gDriveTrainImages = '/content/drive/MyDrive/Datasets/Hackerearth_vehicle_insurance_claim/dataset/trainImages/'
gDriveTestImages = '/content/drive/MyDrive/Datasets/Hackerearth_vehicle_insurance_claim/dataset/testImages/'
gDrivePath = '/content/drive/MyDrive/Datasets/Hackerearth_vehicle_insurance_claim/dataset/'

In [ ]:
df_train = pd.read_csv(gDrivePath+'train.csv')
df_train.head()

,Image_path,Insurance_company,Cost_of_vehicle,Min_coverage,Expiry_date,Max_coverage,Condition,Amount
0,img_4513976.jpg,BQ,41500.0,1037.5,2026-12-03,36142.68,0,0.0
1,img_7764995.jpg,BQ,50700.0,1267.5,2025-07-10,12753.00,1,6194.0
2,img_451308.jpg,A,49500.0,1237.5,2022-08-11,43102.68,0,0.0
3,img_7768372.jpg,A,33500.0,837.5,2022-08-02,8453.00,1,7699.0
4,img_7765274.jpg,AC,27600.0,690.0,2026-05-01,6978.00,1,8849.0


In [ ]:
df_train.isnull().any()

Image_path           False
Insurance_company    False
Cost_of_vehicle       True
Min_coverage          True
Expiry_date          False
Max_coverage          True
Condition            False
Amount                True
dtype: bool

In [ ]:
# df_train['full_image_path'] = df_train['Image_path']
# for i in range(len(df_train['Image_path'])):
#     df_train['full_image_path'][i] = gDriveTrainImages + df_train['Image_path'][i]
# df_train.head()

In [ ]:
# print(df_train['full_image_path'][1])

### Download the headless model

Create the feature extractor. Use `trainable=False` to freeze the variables in the feature extractor layer, so that the training only modifies the new classifier layer.

In [ ]:
# !mkdir -p Hackerearth_vehicle_insurance_claim/feature-vectors

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', pooling='max', input_shape=(299, 299, 3))
preprocessor = tf.keras.applications.inception_resnet_v2.preprocess_input
IMAGE_SIZE = (299, 299)

In [ ]:
# image_path = df_train['full_mage_path'][0]

In [ ]:
# create empty csv with column names
csv_columns = ['image_name']
no_of_extracted_features = 1536

for i in range(1, no_of_extracted_features+1):
  col_name = "{}{}".format('feature_', i)
  csv_columns.append(col_name)

empty_df = pd.DataFrame(columns=csv_columns)
empty_df.to_csv('train_image_features.csv', index=False)

In [ ]:
# returns feature vector
def extract_image_features(image_path, model, preprocessor):
  img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocessor(x)

  features = base_model.predict(x)
  return features

In [ ]:
%%time

for index, row in df_train.iterrows():
  image_path = row['Image_path']
  full_image_path = gDriveTrainImages + image_path
  
  print("working on image : {}".format(image_path))
  
  features = extract_image_features(full_image_path, base_model, preprocessor)
  df = pd.DataFrame(features)
  df.insert(loc=0, column='image_name', value=image_path)
  df.to_csv('train_image_features.csv', mode='a', header=False)


In [ ]:
# %%time
# features = extract_image_features(image_path, base_model, preprocessor)

In [ ]:
# df = pd.DataFrame(features)

In [ ]:
# df.insert(loc=0, column='image_name', value=df_train['Image_path'][1])

In [ ]:
# df.to_csv('train_image_features.csv', mode='a', header=False)

In [ ]:
# df.head()

In [ ]:
df_train_image_features = pd.read_csv('train_image_features.csv')
df_train_image_features.reset_index(inplace = True, drop=True)
df_train_image_features.head()

,image_name,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1497,feature_1498,feature_1499,feature_1500,feature_1501,feature_1502,feature_1503,feature_1504,feature_1505,feature_1506,feature_1507,feature_1508,feature_1509,feature_1510,feature_1511,feature_1512,feature_1513,feature_1514,feature_1515,feature_1516,feature_1517,feature_1518,feature_1519,feature_1520,feature_1521,feature_1522,feature_1523,feature_1524,feature_1525,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535,feature_1536
0,img_4513976.jpg,1.113410,1.195830,0.678696,1.054544,1.535011,0.854014,1.052099,3.466827,4.109343,1.176659,0.710361,2.727606,1.701816,0.676628,1.243950,1.140115,1.558623,1.530318,1.403269,1.848212,1.663212,0.715508,0.331093,0.973033,0.648887,3.715684,0.852375,2.639280,0.776644,4.370374,2.060664,2.183028,0.313798,1.913990,1.074501,1.174085,0.753963,2.989696,1.594213,...,1.362404,3.470226,2.212031,1.512656,2.512051,3.014310,0.479057,1.335440,0.888880,1.828558,2.093749,4.368773,0.466786,2.092361,1.186966,1.036143,2.313602,1.054223,1.080510,4.206156,0.553590,2.627964,1.034772,2.477546,1.833217,1.750974,1.316062,2.266628,2.354492,3.281885,0.213842,1.806247,2.196179,2.928731,0.667834,0.141059,1.338344,1.261466,2.091747,1.223898
1,img_7764995.jpg,1.184981,0.950404,0.987866,1.953136,1.191678,0.670103,1.153660,1.506774,1.693888,1.252585,1.366871,2.116124,0.971451,0.596697,2.564954,0.442221,0.510957,0.000000,4.434040,2.569827,2.152504,0.746451,1.133695,0.000000,1.732929,2.091240,1.270652,1.936253,0.374576,4.306185,1.449873,1.967996,1.509688,0.801316,0.818592,1.381509,1.613760,1.260799,4.557262,...,2.215628,0.534555,1.324353,0.214095,1.976946,0.295753,1.400511,0.438756,1.751667,0.337908,3.021870,0.872242,0.489815,0.815823,0.690202,1.893319,1.511272,0.189446,1.334797,4.581714,0.818972,2.037291,0.306477,4.151568,1.217532,0.594016,0.589151,2.331535,2.162713,0.510852,0.865101,2.179245,0.134113,1.352904,1.690225,0.534583,0.372506,0.675020,0.601140,1.758329
2,img_451308.jpg,0.984364,3.712041,1.342024,1.270731,2.592756,2.304238,1.083133,1.773062,3.126085,3.407647,3.253896,1.742563,0.880907,1.077093,2.729234,1.070258,2.239219,0.777536,3.829961,2.474638,2.073734,0.880441,0.938265,0.194889,0.655862,1.476557,1.733209,2.502393,1.770794,1.850421,0.383996,1.807120,1.117485,0.540778,0.901838,1.331935,1.522768,2.127747,3.239175,...,3.207146,2.156034,1.728536,0.653108,1.860701,1.029206,1.212463,1.302829,2.240293,1.271308,2.979949,2.248640,0.497746,1.644486,3.073427,0.846062,3.447317,0.481213,2.883961,1.777015,1.299833,3.006877,1.301681,2.607335,0.980284,0.722226,0.881499,2.074486,1.128840,2.929870,0.678759,2.807421,0.461782,1.507994,0.231493,1.071761,1.353591,1.181578,0.724902,0.828650
3,img_7768372.jpg,3.665247,1.350056,1.463427,1.000481,1.107578,0.338385,1.853880,2.324681,1.838981,2.579403,0.912768,3.552851,3.520501,1.570992,1.152532,0.670120,1.630700,0.527696,3.928166,3.351275,2.033791,1.283059,0.980372,2.746605,0.352400,1.504906,0.842622,3.030649,1.046014,2.085361,0.966378,2.495072,1.125648,1.367488,1.395767,2.567438,2.488915,2.303958,2.397161,...,1.450805,1.738187,0.945829,3.028120,4.297032,1.605021,1.012109,1.490943,2.094949,1.577143,2.257676,0.688796,1.020116,1.583643,1.610365,1.145513,1.377197,2.338273,0.702233,3.547798,1.608444,2.860399,1.120848,3.461652,4.402316,1.495050,0.793155,4.184282,1.436624,0.643803,0.357418,4.112325,1.071358,1.469693,2.918353,0.943867,0.726660,2.254592,3.938456,1.160831
4,img_7765274.jpg,2.131504,0.675391,0.120818,0.206190,1.625136,0.919565,2.832403,1.702231,2.569179,0.648237,2.386

In [ ]:
# test image feature extraction
df_test = pd.read_csv(gDrivePath+'test.csv')


# create empty csv with column names
csv_columns = ['image_name']
no_of_extracted_features = 1536

for i in range(1, no_of_extracted_features+1):
  col_name = "{}{}".format('feature_', i)
  csv_columns.append(col_name)

empty_df = pd.DataFrame(columns=csv_columns)
empty_df.to_csv('test_image_features.csv', index=False)

CPU times: user 71.9 ms, sys: 3.12 ms, total: 75 ms
Wall time: 604 ms


In [ ]:
%%time
for index, row in df_test.iterrows():
  image_path = row['Image_path']
  full_image_path = gDriveTestImages + image_path
  
  print("working on image : {}".format(image_path))
  
  features = extract_image_features(full_image_path, base_model, preprocessor)
  df = pd.DataFrame(features)
  df.insert(loc=0, column='image_name', value=image_path)
  df.to_csv('test_image_features.csv', mode='a', header=False)

working on image : img_4538519.jpg
working on image : img_7766002.jpg
working on image : img_4637390.jpg
working on image : img_4516108.jpg
working on image : img_4517008.jpg
working on image : img_7766273.jpg
working on image : img_4638351.jpg
working on image : img_4635542.jpg
working on image : img_4638631.jpg
working on image : img_7767742.jpg
working on image : img_4538466.jpg
working on image : img_4536923.jpg
working on image : img_4636354.jpg
working on image : img_4537746.jpg
working on image : img_4635936.jpg
working on image : img_7765021.jpg
working on image : img_4535904.jpg
working on image : img_7768206.jpg
working on image : img_4537049.jpg
working on image : img_7765468.jpg
working on image : img_7766237.jpg
working on image : img_4515135.jpg
working on image : img_4635261.jpg
working on image : img_4637068.jpg
working on image : img_4514590.jpg
working on image : img_4635133.jpg
working on image : img_463748.jpg
working on image : img_7766331.jpg
working on image : im

In [ ]:
df_test_image_features = pd.read_csv('test_image_features.csv')
df_test_image_features.reset_index(inplace = True, drop=True)
df_test_image_features.head()

,image_name,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_1497,feature_1498,feature_1499,feature_1500,feature_1501,feature_1502,feature_1503,feature_1504,feature_1505,feature_1506,feature_1507,feature_1508,feature_1509,feature_1510,feature_1511,feature_1512,feature_1513,feature_1514,feature_1515,feature_1516,feature_1517,feature_1518,feature_1519,feature_1520,feature_1521,feature_1522,feature_1523,feature_1524,feature_1525,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535,feature_1536
0,img_4538519.jpg,0.973620,0.995891,2.086252,0.754458,0.183079,1.328674,0.646375,2.154040,1.111717,0.972564,0.823164,0.796428,0.129889,0.886301,1.117819,0.615297,0.215030,1.084852,1.213631,1.723843,2.062928,2.532325,0.401523,0.507209,1.460627,1.394074,1.781820,1.221760,0.963381,3.043749,0.960874,2.597368,0.962506,0.139617,0.159631,1.078381,0.999952,0.839636,2.016972,...,3.127630,1.375920,0.103106,0.779019,1.153867,1.159208,0.650050,1.038933,1.910786,0.098227,2.771692,0.700567,1.492984,1.797701,1.729574,0.914373,0.065630,0.530929,1.624197,0.808059,0.582138,2.105141,0.240400,1.633089,0.578716,0.851595,0.639973,1.401339,0.837881,0.676245,0.935792,1.758861,0.227776,0.257120,0.518423,0.434983,1.548907,0.756545,0.162454,0.379081
1,img_7766002.jpg,0.999409,1.843040,2.522187,0.412937,2.507851,0.611475,1.317616,1.446599,0.731480,2.777624,0.819642,2.747401,2.333248,1.431331,3.020208,1.103773,2.692001,1.237704,2.280685,1.744851,1.239900,0.705429,0.263593,0.671913,1.227257,1.802247,1.057694,2.600135,0.991552,1.567271,1.203328,1.294116,2.238398,0.716651,0.834616,3.514566,2.001752,1.541599,1.961557,...,2.635256,1.341455,1.065197,1.140869,1.320284,0.989244,1.105877,0.570188,4.379395,0.782135,3.392764,1.437731,0.530782,1.248665,2.564096,1.836590,2.498237,2.367906,1.789405,4.599102,0.966702,2.893390,0.431540,4.815420,4.175478,1.364682,2.098490,1.602103,3.849216,1.977615,0.101044,1.710837,2.896975,2.110524,1.017958,0.000000,2.391906,1.722817,1.690435,0.991267
2,img_4637390.jpg,2.246830,1.778973,3.412202,1.156048,1.403798,2.298196,1.281496,2.512424,1.458304,2.440149,1.988262,2.314021,1.845660,3.229697,5.362529,2.853629,1.223527,1.597957,3.423955,3.114634,1.211126,1.597503,2.472815,0.457019,2.102471,2.619133,4.224362,4.143013,2.455480,2.148308,1.094683,3.838666,1.757173,1.647232,2.549886,1.404592,1.904021,3.207250,2.206167,...,3.742384,1.136305,1.242690,1.748059,2.857538,1.943526,0.944426,1.661380,1.467122,2.237052,1.993747,0.278631,1.346481,3.712535,2.018367,0.063781,0.406501,2.117756,3.456734,3.222384,2.061197,3.888697,3.205869,0.875596,1.108164,2.919541,0.795669,7.082536,3.192329,1.165484,0.987660,3.197035,1.961972,1.795293,1.651497,1.461264,1.013859,1.497607,2.664895,1.692402
3,img_4516108.jpg,0.888304,1.757760,1.971983,3.466796,0.297903,0.293281,1.652176,1.830663,2.614499,2.392623,2.528785,1.893961,0.664629,2.120634,2.797562,2.362875,3.029844,2.160243,1.452202,3.317678,1.154193,2.794878,1.306739,0.646381,1.708374,2.831292,1.998458,4.443255,1.338708,2.749506,2.115950,1.776223,1.247378,0.426952,1.497869,0.794166,2.136837,2.078762,1.856165,...,2.585384,1.277332,2.322298,0.727883,3.454577,1.945839,0.334075,0.943366,2.176439,2.471758,2.453000,0.678553,0.100992,2.337682,2.124885,0.824984,1.638389,1.377770,1.430217,2.786345,1.794773,2.172104,0.769504,2.793397,1.324917,2.353707,0.925193,1.848623,2.320792,2.707424,0.731340,3.271449,1.268554,1.471442,1.086145,0.413754,2.556309,0.323846,0.813326,1.539042
4,img_4517008.jpg,2.972563,1.287300,2.802837,3.145813,0.914180,2.403655,3.380828,4.743644,2.448464,2.621379,0.82